In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Load the Test dataset

In [ ]:
sentiment_text = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
sentiment_text.head()

In [ ]:
sentiment_text[sentiment_text.textID == "87daa62920"]

In [ ]:
sentiment_text["text"] = sentiment_text["text"].str.strip()

#### Prepare the data file

In [ ]:
output_file = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/sample_submission.csv")
output_file = output_file[["textID"]]
output_file.head()

In [ ]:
output_file.shape

##### First let's prepare for the neural sentiment

In [ ]:
neutral_sentiment = sentiment_text[sentiment_text.sentiment == 'neutral']
neutral_sentiment.head()

In [ ]:
neutral_sentiment = neutral_sentiment[["textID","text"]]
neutral_sentiment.columns = ["textID","selected_text"]
neutral_sentiment.head()

In [ ]:
output_file = pd.merge(output_file, neutral_sentiment,how='left',on="textID")
output_file.shape

In [ ]:
output_file.head()

#### Now get the results for Positive and Negative sentiments

In [ ]:
sentiment_text.shape

In [ ]:
pos_neg_data = sentiment_text[sentiment_text.sentiment != 'neutral']
pos_neg_data["idx"] = pos_neg_data.index
pos_neg_data.head()

Load the results data

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [ ]:
test_result = pd.read_csv("/kaggle/input/model-training/test_result_1.csv")
test_result = test_result[["idx","predicted_text","unks"]]
test_result.head()

In [ ]:

pos_neg_data = pd.merge(pos_neg_data,test_result,how='left',on='idx')
pos_neg_data.head()

In [ ]:
pos_neg_data.iloc[1600:1610]

In [ ]:
pos_neg_data = pos_neg_data[["textID","predicted_text"]]
#pos_neg_data.columns = ["textID","selected_text"]
pos_neg_data.shape

##### Combine all the data together

In [ ]:
output_file.shape

In [ ]:
output_file = pd.merge(output_file,pos_neg_data,how='left',on='textID')
output_file.head()

In [ ]:
output_file["selected_text"] = output_file["selected_text"].fillna(output_file["predicted_text"])

In [ ]:
output_file[output_file.selected_text != output_file.selected_text].shape

In [ ]:
output_file[output_file.predicted_text != output_file.predicted_text].shape

In [ ]:
output_file = output_file[["textID","selected_text"]]
output_file["selected_text"] = output_file["selected_text"].str.strip()
output_file.shape

In [ ]:
output_file = pd.merge(output_file, sentiment_text[["textID","text"]],how='left',on='textID')
output_file.head()

In [ ]:
output_file["selected_text"] = output_file["selected_text"].fillna(output_file["text"])
output_file.head()

In [ ]:
output_file = output_file[["textID","selected_text"]]
output_file["selected_text"] = output_file["selected_text"].str.strip()
output_file.head()

In [ ]:
output_file.to_csv("/kaggle/working/submission.csv",index=False)